In [73]:
import openai,os,requests,json,re
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [74]:
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

## Prompts

In [75]:
## Initial Prompt
initial_str = """\
## Output Format
```json
{output_format}
```

## Input
{text}

## Instruction
[Input]はあるWebページのテキストの一部です。\
[Output Format]に従って、json.loads()関数でパースできる形で出力してください。\
[Input]から読み取れないデータは出力に含めないでください。
"""

## Refine Prompt
refine_str = """\
## Output Format
```json
{output_format}
```

## Data
{data}

## Text
{text}


## Instruction
あなたのタスクは、最終的なデータを生成することです。\
現時点でのデータは[Data]です。
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
[Output Format]に従って、json.loads()関数でパースできる形で出力してください。\
"""
initial_prompt_template = ChatPromptTemplate.from_template(initial_str)
print('initial:',initial_prompt_template.messages[0].prompt.input_variables)
refine_prompt_template = ChatPromptTemplate.from_template(refine_str)
print('refine:',refine_prompt_template.messages[0].prompt.input_variables)

initial: ['output_format', 'text']
refine: ['data', 'output_format', 'text']


## ヘルパー関数

In [76]:
def load_doc(url):
    response = requests.get(url)
    with open('temp.html','w') as f:
        f.write(response.text)
    loader = UnstructuredHTMLLoader("temp.html")
    data = loader.load()
    return data

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3072,
    chunk_overlap  = 512,
    length_function = len,
)

chat = ChatOpenAI(temperature=0.0)

def extract_json(text,debug=False):
    patterns = [
        {"pattern": r'```json\n(.*?)```', "group": 1},
        {"pattern": r'\{.*\}', "group": 0},
    ]
    for p in patterns:
        match = re.search(p["pattern"], text, re.DOTALL)
        if match:
            json_string = match.group(p["group"])
            if debug:
                print('json_string:',json_string,sep='\n')
            data_json = json.loads(json_string)
            return data_json
    print('Error:Could not json extracted:',text)
    return {}

## メイン関数

In [85]:
def extact_json_from_url(url,data_format,debug=False):
    doc = load_doc(url)
    docs = text_splitter.split_documents(doc)
    data = {}
    for i,doc in enumerate(docs):
        text = doc.page_content
        if debug:
            print(i+1,'-'*30)
            print(text.replace('\n','').replace(' ','')[:100],'...')
            print('...',text.replace('\n','').replace(' ','')[-100:])
        if i == 0:
            messages = initial_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        else:
            messages = refine_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text,
                data=json.dumps(data,indent=2,ensure_ascii=False)
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        if debug:
            print(json.dumps(data,ensure_ascii=False))
    data['url'] = url
    return data

## レシピ

In [86]:
recipe_format = {
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
            "g": "int: 分量表記から推定した重量(gram)"
        }
    ],
    "steps": {
        "str: 手順番号":"str: 手順の説明"
    }
}

In [87]:
# url = "https://park.ajinomoto.co.jp/recipe/card/705645/"
url = "https://www.lettuceclub.net/recipe/dish/14360/"

In [88]:
# data = extact_json_from_url(url,recipe_format)
# print(json.dumps(data,indent=2,ensure_ascii=False))

## イベント

In [89]:
event_format = {
    'title': 'str: イベント名',
    'organizer': 'str: 主催者',
    'date': ['str: 日付。形式はYYYY/MM/DD'],
    'location': 'str: 開催場所',
    'schedule': ['str: スケジュール。形式はYYYY/MM/DD HH:MM~HH:MM（補足）'],
    'description': 'str: イベント概要。400字以内',
}

In [90]:
# url = "https://food-innovation.co/sksjapan/sksj2023/"
url = "https://www.nagoya-info.jp/event/detail/404/"

In [91]:
data = extact_json_from_url(url,event_format,debug=False)
print(json.dumps(data,indent=2,ensure_ascii=False))

{
  "title": "オアシス２１iセンター体験イベント＜有松・鳴海絞り染め＞",
  "organizer": "オアシス２１iセンター",
  "date": [
    "2023/06/12"
  ],
  "location": "オアシス２１iセンター前（オアシス２１地下1階）",
  "schedule": [
    "2023/06/12 10:00~18:00",
    "2023/06/12 30分"
  ],
  "description": "名古屋をもっと楽しみたい時は、オアシス２１で伝統産業を体験しませんか。\n\n「有松・鳴海絞り染め」「名古屋黒紋付染め」「名古屋友禅」など、名古屋には古くから染めものが根付いています。\nオアシス２１iセンターでは染めの楽しさを感じてもらえる「有松・鳴海絞り染め」の体験イベントを毎月開催しています。\n\n小さなお子様も大歓迎ですので、お気軽にご参加ください。\n\n英語対応も可能です。",
  "url": "https://www.nagoya-info.jp/event/detail/404/"
}
